In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

#### Fetch xml sitemap

In [3]:
sitemap = pd.read_xml("https://www.rtbf.be/site-map/articles.xml")
sitemap.head()

,loc,lastmod,changefreq,news,image
0,https://www.rtbf.be/article/taux-d-interet-des...,2023-08-01T06:46:04.000Z,hourly,NaN,NaN
1,https://www.rtbf.be/article/coupe-du-monde-fem...,2023-08-01T06:30:42.000Z,hourly,NaN,NaN
2,https://www.rtbf.be/article/baraki-est-bientot...,2023-08-01T06:30:00.000Z,hourly,NaN,NaN
3,https://www.rtbf.be/article/james-webb-les-plu...,2023-08-01T06:05:00.000Z,hourly,NaN,NaN
4,https://www.rtbf.be/article/4-conseils-pour-ge...,2023-08-01T06:00:06.000Z,hourly,NaN,NaN


#### Keep only two first columns and change their name

In [4]:
df = sitemap.drop(["changefreq", "news", "image"], axis=1)
df.rename(columns={"loc": "source_url", "lastmod": "date"}, inplace=True)
df.head()

,source_url,date
0,https://www.rtbf.be/article/taux-d-interet-des...,2023-08-01T06:46:04.000Z
1,https://www.rtbf.be/article/coupe-du-monde-fem...,2023-08-01T06:30:42.000Z
2,https://www.rtbf.be/article/baraki-est-bientot...,2023-08-01T06:30:00.000Z
3,https://www.rtbf.be/article/james-webb-les-plu...,2023-08-01T06:05:00.000Z
4,https://www.rtbf.be/article/4-conseils-pour-ge...,2023-08-01T06:00:06.000Z


#### Keep only date in 'date' column

In [5]:
df["date"].replace({r"T.+": ""}, inplace=True, regex=True)
df.head()

,source_url,date
0,https://www.rtbf.be/article/taux-d-interet-des...,2023-08-01
1,https://www.rtbf.be/article/coupe-du-monde-fem...,2023-08-01
2,https://www.rtbf.be/article/baraki-est-bientot...,2023-08-01
3,https://www.rtbf.be/article/james-webb-les-plu...,2023-08-01
4,https://www.rtbf.be/article/4-conseils-pour-ge...,2023-08-01


In [6]:
urls = df["source_url"]
urls.head()

0    https://www.rtbf.be/article/taux-d-interet-des...
1    https://www.rtbf.be/article/coupe-du-monde-fem...
2    https://www.rtbf.be/article/baraki-est-bientot...
3    https://www.rtbf.be/article/james-webb-les-plu...
4    https://www.rtbf.be/article/4-conseils-pour-ge...
Name: source_url, dtype: object

In [7]:
article_one = urls[0]
print(article_one)

https://www.rtbf.be/article/taux-d-interet-des-comptes-d-epargne-remboursement-des-lunettes-vignette-suisse-ce-qui-change-ce-1er-aout-11234097


In [8]:
response = requests.get(article_one)
response.text

'<!DOCTYPE html><html lang="fr" dir="ltr"><head><meta charSet="utf-8"/><meta name="viewport" content="initial-scale=1.0, width=device-width, minimum-scale=1"/><meta property="al:android:url" content="rtbf://applinks/article/taux-d-interet-des-comptes-d-epargne-remboursement-des-lunettes-vignette-suisse-ce-qui-change-ce-1er-aout-11234097"/><meta property="al:android:app_name" content="RTBF"/><meta property="al:android:package" content="be.rtbf"/><meta property="al:web:url" content="https://rtbf.be"/><link rel="preconnect" href="https://ds.static.rtbf.be"/><link rel="dns-prefetch" href="https://ds.static.rtbf.be"/><link rel="preconnect" href="https://ds1.static.rtbf.be"/><link rel="dns-prefetch" href="https://ds1.static.rtbf.be"/><link rel="preconnect" href="https://static-oaos.rtbf.be"/><link rel="dns-prefetch" href="https://static-oaos.rtbf.be"/><link rel="preconnect" href="https://sgc.static.rtbf.be"/><link rel="dns-prefetch" href="https://sgc.static.rtbf.be"/><link rel="preconnect" h

In [10]:
soup = bs(response.content, "html.parser")
print(soup)

<!DOCTYPE html>
<html dir="ltr" lang="fr"><head><meta charset="utf-8"/><meta content="initial-scale=1.0, width=device-width, minimum-scale=1" name="viewport"/><meta content="rtbf://applinks/article/taux-d-interet-des-comptes-d-epargne-remboursement-des-lunettes-vignette-suisse-ce-qui-change-ce-1er-aout-11234097" property="al:android:url"/><meta content="RTBF" property="al:android:app_name"/><meta content="be.rtbf" property="al:android:package"/><meta content="https://rtbf.be" property="al:web:url"/><link href="https://ds.static.rtbf.be" rel="preconnect"/><link href="https://ds.static.rtbf.be" rel="dns-prefetch"/><link href="https://ds1.static.rtbf.be" rel="preconnect"/><link href="https://ds1.static.rtbf.be" rel="dns-prefetch"/><link href="https://static-oaos.rtbf.be" rel="preconnect"/><link href="https://static-oaos.rtbf.be" rel="dns-prefetch"/><link href="https://sgc.static.rtbf.be" rel="preconnect"/><link href="https://sgc.static.rtbf.be" rel="dns-prefetch"/><link href="https://www.

#### Find article title

In [93]:
title = soup.find("h1").text
print(title)

200 personnes à l’ouverture de la crypte royale pour rendre hommage au Roi Baudouin


#### Find article title function

In [94]:
def find_article_title(url: str) -> str:
    response = requests.get(url)
    soup = bs(response.content, "html.parser")
    article_title = soup.find("h1").text
    return article_title

In [95]:
print(find_article_title(article_one))

200 personnes à l’ouverture de la crypte royale pour rendre hommage au Roi Baudouin


#### Find article text

In [12]:
paragraphs = soup.find_all("p", attrs={"class": None})
article_text = ""
for paragraph in paragraphs:
    article_text += paragraph.text
    article_text += "\n"
print(article_text)

article : Ce mardi 1er août 2023 est marqué notamment par des changements qui concernent les taux d’intérêt appliqués par les banques, le remboursement des verres de lunette et la vignette pour circuler sur les autoroutes en Suisse.
Voici quelques-uns de ces principaux changements :
Les clients de KBC et CBC bénéficieront d’un taux de base et d’une prime de fidélité plus élevés sur les comptes d’épargne réglementés à partir du 1er août.
Tous les clients particuliers des deux banques jouiront d’un taux de base et d’une prime de fidélité plus élevés sur leurs comptes Start2Save et Start2Save4, qui bénéficient d’un taux d’intérêt plus élevé, ainsi que sur leurs comptes d’épargne ordinaires.
Le taux d’intérêt du compte d’épargne ordinaire est adapté comme suit : le taux de base passe de 0,35% à 0,45% et la prime de fidélité pour l’argent laissé sur le compte pendant au moins un an sans interruption évolue de 0,25% à 0,45%. Soit un taux d’intérêt total de 0,90%.
Pour Start2Save et Start2Sav

#### Find article text function

In [97]:
def find_article_text(url: str) -> str:
    response = requests.get(url)
    soup = bs(response.content, "html.parser")
    paragraphs = soup.find_all("p", attrs={"class": None})
    article_text = ""
    for paragraph in paragraphs:
        article_text += paragraph.text
    return article_text

In [98]:
print(find_article_text(article_one))

Ils étaient environ deux cents à attendre que les grilles de la crypte royale de Notre-Dame de Laeken s’ouvrent enfin ce lundi 31 août. Derrière elles, ce sont les rois et les reines de notre pays qui reposent dans une grande et solennelle salle ornée de marbres polychromes. Face à l’impressionnant monument érigé pour Léopold 1er et la Reine Louise-Marie, un tombeau plus modeste abrite les sépultures de Baudouin et Fabiola. Devant elle, une élégante gerbe de roses blanches relevées des couleurs du drapeau belge a été disposée.À 14 heures, les premiers visiteurs sont enfin autorisés à entrer. Devant le tombeau, les discussions s’arrêtent, certains osent sortir leurs téléphones pour prendre une photo, d’autres regardent avec insistance le cliché en noir et blanc des deux souverains pris peu de temps après leur mariage.Dans l’assistance, des Wallons, des Bruxellois et des Flamands sont réunis. Un couple d’Anversois est venu en voiture pour l’occasion. Des passionnés d’histoire qui saluent